In [16]:
import os
import pandas as pd
import re
import copy

cur_dir = os.getcwd()

def find_index_first(list_to_search,search_text):
    for i, string in enumerate(list_to_search):
        if len(string) > len(search_text):
            if string[:len(search_text)] == search_text:
                return i
    return -1   # Return -1 if no string found with first letter 'n'


summary_dict = dict()
score = {"distance" : 0, "time" : 0}


for dirPath, dirNames, fileNames in os.walk(cur_dir):
    for f in fileNames:
        # print(f)
        split_list = re.split(r'[_.]', f)
        # print(split_list)
        
        
        n_idx = find_index_first(split_list,"n")
        m_idx = find_index_first(split_list,"m")
        seed_idx = find_index_first(split_list,"seed")
        cmax_idx = find_index_first(split_list,"cmax")
        cmin_idx = find_index_first(split_list,"cmin")
        source_idx = find_index_first(split_list,"source")
        network_idx = n_idx-1
        method_list = split_list[:network_idx]
        cond_list = split_list[network_idx:cmin_idx+1]
        method = "_".join(method_list)
        cond = "_".join(cond_list)

        # print(method)
        # print(cond)

        if cond not in summary_dict:
            summary_dict[cond] = dict()
        if method not in summary_dict[cond]:
            summary_dict[cond][method] = copy.deepcopy(score)

        file_path = os.path.join(dirPath, f)
        file = open(file_path, 'r')
        text = []
        for line in file:
            # text.append(line)
            if line[:3] == "Sum":
                distance = re.search(r"Sum of distances:\s*([\d.]+)", line).group(1)
                summary_dict[cond][method]["distance"] = distance
            if line[:3] == "Run":
                time = re.search(r"Running time of SP computation:\s*([\d.]+)", line).group(1)
                summary_dict[cond][method]["time"] = time

        file.close

print(summary_dict)

{'spgrid_n2000_m12000_seed2_cmax100_cmin10': {'My_spheap_long': {'distance': '2948829103852', 'time': '1813.000000'}, 'My_BF_deque_long': {'distance': '2948829103852', 'time': '598.000000'}, 'dikh': {'distance': '2948829103852', 'time': '3.37'}, 'My_spdial_long': {'distance': '2948829103852', 'time': '166097.000000'}, 'bf': {'distance': '2948829103852', 'time': '17.20'}, 'My_BF_queue_long': {'distance': '2948829103852', 'time': '8546.000000'}, 'dikb': {'distance': '2948829103852', 'time': '1.57'}, 'bfp': {'distance': '2948829103852', 'time': '15.07'}}, 'spgrid_n100_m500_seed3_cmax10000_cmin100': {'bfp': {'distance': '1600130502', 'time': '0.07'}, 'dikb': {'distance': '1600130502', 'time': '0.00'}, 'bf': {'distance': '1600130502', 'time': '0.10'}, 'My_spdial_long': {'distance': '1600130502', 'time': '178.000000'}, 'My_spheap': {'distance': '1600130502', 'time': '3.000000'}, 'dikh': {'distance': '1600130502', 'time': '0.00'}, 'My_BF_deque': {'distance': '1600130502', 'time': '5.000000'},

In [18]:
df = pd.DataFrame.from_dict({(i,j): summary_dict[i][j] 
                             for i in summary_dict.keys() 
                             for j in summary_dict[i].keys()},
                            orient='index')
display(df)
final_summary_path = os.path.join(cur_dir,"summary_table.xlsx")
df.to_excel(final_summary_path)

distance  \
spgrid_n2000_m12000_seed2_cmax100_cmin10 My_spheap_long    2948829103852   
                                         My_BF_deque_long  2948829103852   
                                         dikh              2948829103852   
                                         My_spdial_long    2948829103852   
                                         bf                2948829103852   
...                                                                  ...   
sprand_n2000_m12000_seed2_cmax100_cmin10 My_BF_queue              321959   
                                         My_BF_deque              321959   
                                         dikh                     321959   
                                         My_spheap                321959   
                                         summarize                     0   

                                                                    time  
spgrid_n2000_m12000_seed2_cmax100_cmin10 My_spheap_long      1813.000000  
                                         My_BF_deque_long     598.000000  
                                         dikh                       3.37  
                                         My_spdial_long    166097.000000  
                                         bf                        17.20  
...                                                                  ...  
sprand_n2000_m12000_seed2_cmax100_cmin10 My_BF_queue           13.000000  
                                         My_BF_deque           12.000000  
                                         dikh                       0.00  
                                         My_spheap              0.000000  
                                         summarize                     0  

[153 rows x 2 columns]